In [1]:
import os
%pwd

'/Users/slackroo/Data_science/GitHub/kidney_disease_classification_deeplearning/research'

In [2]:
os.chdir("../")
%pwd

'/Users/slackroo/Data_science/GitHub/kidney_disease_classification_deeplearning'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]= "https://dagshub.com/slackroo/kidney_disease_classification_deeplearning.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="slackroo"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e383057a1f050e672c874aef175d897f88ad2ea2"

In [4]:
import  tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

2023-10-09 00:23:25.331730: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/slackroo/Data_science/GitHub/kidney_disease_classification_deeplearning/venv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
# workflow entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [6]:
# config manager workflow
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories, save_json
import tensorflow as tf

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/slackroo/kidney_disease_classification_deeplearning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [7]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [8]:
class Evaluation:
    def __init__(self, config= EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [9]:
# workflow pipeline

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2023-10-09 00:23:29,637: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-09 00:23:29,641: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-09 00:23:29,642: INFO: common: created directory at: artifacts]
Found 204 images belonging to 2 classes.


2023-10-09 00:23:30.515494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 20s 3s/step - loss: 20.9755 - accuracy: 0.7059
[2023-10-09 00:23:50,436: INFO: common: json file saved at: scores.json]


2023/10/09 00:23:53 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-10-09 00:23:54,819: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/6d/ydlrjs7x3_n6lhh19hn27v200000gn/T/tmpodddf811/model/data/model/assets
[2023-10-09 00:23:55,687: INFO: builder_impl: Assets written to: /var/folders/6d/ydlrjs7x3_n6lhh19hn27v200000gn/T/tmpodddf811/model/data/model/assets]


/Users/slackroo/Data_science/GitHub/kidney_disease_classification_deeplearning/venv/lib/python3.8/site-packages/_distutils_hack/__init__.py:36: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/10/09 00:25:20 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.
